### Investigate the convertion from CARMA params to Celerite terms

<br>**Author(s):** Weixiang Yu
<br>**Last run:** 06-17-20
<br>**Short description:** Two tasks: 1. Examine the built in CARMA solver of Celertie; 2. If not working, develope a new convertion fuction.

In [1]:
# import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys

# see if local stores mpl style, else use from src
try:
    plt.style.use('yu_basic')
except:
    mpl.rc_file('../../src/vis/mpl/yu_basic.rc')

pd.set_option('display.max_columns', 999)
%matplotlib inline

In [2]:
import celerite
from celerite import GP, terms
from celerite.solver import get_kernel_value, CARMASolver

### 1. Test Celerite CARMA Solver
The first step is to see whether the built in CARMASolver give the correct convertion by examing the the ACF of a input CARMA model. Given an overdamped 2nd order CARMA model (two real roots), we know the ACF pairs must have opposite signs. And thus the corresponding Celerite real terms must also have opposite signs. 
<br> Here we are using a CARMA(2,0) model, with AR parameters: [2, 0.8] and $\sigma$ = 1

In [3]:
# compute the ACF of the input model
arroots = np.roots([1, 2, 0.8])
sigma = 1
print (f'AR roots: {arroots}')

AR roots: [-1.4472136 -0.5527864]


In [4]:
r1, r2 = arroots
acf_1 = sigma**2/(-2*r1*(r2-r1)*(r2+r1))
acf_2 = sigma**2/(-2*r2*(r1-r2)*(r1+r2))
print(f'ACF for root 1: {acf_1}')
print(f'ACF for root 2: {acf_2}')

ACF for root 1: -0.19313562148434218
ACF for root 2: 0.5056356214843422


#### Now see what Celerite returns

In [55]:
carma_solver = CARMASolver(0, np.log(np.array([0.8, 2])), np.array([]))
carma_solver.get_celerite_coeffs()

(array([ 0.50563562, -0.19313562]),
 array([0.5527864, 1.4472136]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64))

<span style='color:red'>
The built in CARMA solver gives correct `Celerite` terms with two catches: 1. Both AR and MA parameters must be provided in the log scale. 2. The order of AR params must follow the index in the Kelly et al. 2014 paper. 

<br>! See section 2.3 for an update on more complex models. (Celerite CARMA solver has bugs)
</span>

### 2. Hard Coded CARMA ACF + More tests for Celerite CARMA solver

In [6]:
# construct/test using CARMA(2,1), define in Kasliwal's notation and add to Kelly's
arparam = np.array([2, 0.8])
maparam = np.array([1])

sigma = maparam[0]
n_maparam = np.array([x/sigma for x in maparam])

In [7]:
# get roots
arroots = np.roots(np.append([1], arparam))
maroots = np.roots(n_maparam[::-1])
print(f'AR roots: {arroots}')
print(f'MA roots: {maroots}')

AR roots: [-1.4472136 -0.5527864]
MA roots: []


In [8]:
# variable for order
p = len(arparam)
q = len(maparam) -1

#### 2.1 Vectorized CARMA ACF

In [9]:
# get acf in a vectorized way
num_0 = 0
num_1 = 0
denum = -2*arroots.real

for k in range(q+1):
    num_0 += maparam[k]*np.power(arroots, k)
    num_1 += maparam[k]*np.power(-arroots, k)

for j in range(1, p):
    root_idx = np.arange(p)
    root_k = arroots[np.roll(root_idx, j)]
    print(root_k)
    denum *= (root_k - arroots)*(np.conj(root_k) + arroots)

[-0.5527864 -1.4472136]


In [10]:
print(f'ACF for CARMA(2,0): {num_0*num_1/denum}')

ACF for CARMA(2,0): [-0.19313562  0.50563562]


#### 2.2 Put ACF into a function

In [36]:
def acf(arparam, maparam):
    p = len(arparam)
    q = len(maparam)-1    
    sigma = maparam[0]
    
    # MA param into Kell's notation
    arparam = np.array(arparam)
    maparam = np.array([x/sigma for x in maparam])
    
    # get roots
    arroots = np.roots(np.append([1], arparam))
#     maroots = np.roots(n_maparam[::-1])
    
    # init acf product terms
    num_left = 0
    num_right = 0
    denom = -2*arroots.real + np.zeros_like(arroots)*1j 
    
    for k in range(q+1):
        num_left += maparam[k]*np.power(arroots, k)
        num_right += maparam[k]*np.power(-arroots, k)

    for j in range(1, p):
        root_idx = np.arange(p)
        root_k = arroots[np.roll(root_idx, j)]
        denom *= (root_k - arroots)*(np.conj(root_k) + arroots)

    return sigma**2*num_left*num_right/denom

def acf20(arparam, maparam):
    
    assert len(arparam) == 2
    assert len(maparam) == 1
    
    sigma = maparam[0]
    maparam = np.array([x/sigma for x in maparam])
    r_1, r_2 = np.roots(np.append([1], arparam))
    
    acf_1 = sigma**2/(-2*r_1.real*(r_2-r_1)*(np.conj(r_2)+r_1))
    acf_2 = sigma**2/(-2*r_2.real*(r_1-r_2)*(np.conj(r_1)+r_2))
    
    return np.array([acf_1, acf_2])

def acf21(arparam, maparam):
    
    assert len(arparam) == 2
    assert len(maparam) == 2
    
    sigma = maparam[0]
    maparam = np.array([x/sigma for x in maparam])
    r1, r2 = np.roots(np.append([1], arparam))
    
    acf_1 = sigma**2*(1-maparam[1]**2*r1**2)/(-2*r1.real*(r2-r1)*(np.conj(r2)+r1))
    acf_2 = sigma**2*(1-maparam[1]**2*r2**2)/(-2*r2.real*(r1-r2)*(np.conj(r1)+r2))
    
    return np.array([acf_1, acf_2])

def acf30(arparam, maparam):
    
    assert len(arparam) == 3
    assert len(maparam) == 1
    
    sigma = maparam[0]
    maparam = np.array([x/sigma for x in maparam])
    r1, r2, r3 = np.roots(np.append([1], arparam))
    
    acf_1 = sigma**2/(-2*r1.real*(r2-r1)*(np.conj(r2)+r1)*(r3-r1)*(np.conj(r3)+r1))
    acf_2 = sigma**2/(-2*r2.real*(r1-r2)*(np.conj(r1)+r2)*(r3-r2)*(np.conj(r3)+r2))
    acf_3 = sigma**2/(-2*r3.real*(r1-r3)*(np.conj(r1)+r3)*(r2-r3)*(np.conj(r2)+r3))
    
    return np.array([acf_1, acf_2, acf_3])

def Kali2Pack(arparam, maparam):

    sigma = maparam[0]
    arparam = arparam[::-1]
    maparam = [x/sigma for x in maparam[1:]]

    return sigma, arparam, maparam

#### 2.3 Construct more test examples

**CARMA(2,0)**

In [95]:
# CARMA (2,0)
ar1 = np.array([2, 1.1])
ma1 = np.array([0.5])

ar2 = np.array([2, 0.8])
ma2 = np.array([2])

In [96]:
print('Complex CARMA(2,0)\n')
print(f'Roots: {np.roots(np.append([1], ar1))}')
print(f'Hard Code ACF: {acf20(ar1, ma1)}')
print(f'Vector ACF: {acf(ar1, ma1)}')

# check terms given by CARMA solver
sigma, arpack, mapack = Kali2Pack(ar1, ma1)

print('\nTerms by CARMA Solver:')
carma_solver = CARMASolver(np.log(sigma), np.log(arpack), np.log(mapack))
carma_solver.get_celerite_coeffs()

Complex CARMA(2,0)

Roots: [-1.+0.31622777j -1.-0.31622777j]
Hard Code ACF: [0.02840909-0.08983743j 0.02840909+0.08983743j]
Vector ACF: [0.02840909-0.08983743j 0.02840909+0.08983743j]

Terms by CARMA Solver:


(array([], dtype=float64),
 array([], dtype=float64),
 array([0.05681818]),
 array([-0.17967487]),
 array([1.]),
 array([-0.31622777]))

In [97]:
print('Real CARMA(2,0)\n')
print(f'Roots: {np.roots(np.append([1], ar2))}')
print(f'Hard Code ACF: {acf20(ar2, ma2)}')
print(f'Vector ACF: {acf(ar2, ma2)}')

# check terms given by CARMA solver
sigma, arpack, mapack = Kali2Pack(ar2, ma2)

print('\nTerms by CARMA Solver:')
carma_solver = CARMASolver(np.log(sigma), np.log(arpack), np.log(mapack))
carma_solver.get_celerite_coeffs()

Real CARMA(2,0)

Roots: [-1.4472136 -0.5527864]
Hard Code ACF: [-0.77254249  2.02254249]
Vector ACF: [-0.77254249-0.j  2.02254249+0.j]

Terms by CARMA Solver:


(array([ 2.02254249, -0.77254249]),
 array([0.5527864, 1.4472136]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64))

**CARMA(2,1)**

In [128]:
ar3 = np.array([2, 0.8])
ma3 = np.array([1, 0.5])

ar4 = np.array([2, 1.2])
ma4 = np.array([1, 2])

In [138]:
print('Real CARMA(2,1)\n')
print(f'Roots: {np.roots(np.append([1], ar3))}')
print(f'Hard Code ACF: {acf21(ar3, ma3)}')
print(f'Vector ACF: {acf(ar3, ma3)}')

# check terms given by CARMA solver
sigma, arpack, mapack = Kali2Pack(ar3, ma3)

print('\nTerms by CARMA Solver:')
carma_solver = CARMASolver(np.log(sigma), np.log(arpack), np.log(mapack))
carma_solver.get_celerite_coeffs()

Real CARMA(2,1)

Roots: [-1.4472136 -0.5527864]
Hard Code ACF: [-0.0920085  0.4670085]
Vector ACF: [-0.0920085-0.j  0.4670085+0.j]

Terms by CARMA Solver:


(array([-0.11239837,  1.42489837]),
 array([0.5527864, 1.4472136]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64))

In [100]:
print('Complex CARMA(2,1)\n')
print(f'Roots: {np.roots(np.append([1], ar4))}')
print(f'Hard Code ACF: {acf21(ar4, ma4)}')
print(f'Vector ACF: {acf(ar4, ma4)}')

# check terms given by CARMA solver
sigma, arpack, mapack = Kali2Pack(ar4, ma4)

print('\nTerms by CARMA Solver:')
carma_solver = CARMASolver(np.log(sigma), np.log(arpack), np.log(mapack))
carma_solver.get_celerite_coeffs()

Complex CARMA(2,1)

Roots: [-1.+0.4472136j -1.-0.4472136j]
Hard Code ACF: [0.60416667+0.88511024j 0.60416667-0.88511024j]
Vector ACF: [0.60416667+0.88511024j 0.60416667-0.88511024j]

Terms by CARMA Solver:


(array([], dtype=float64),
 array([], dtype=float64),
 array([0.27083333]),
 array([-0.32609325]),
 array([1.]),
 array([-0.4472136]))

<span style='color:red'>
The CARMA(2,1) test exposes a possible bug in the CARMA solver where it takes the log of negative numbers
</span>

**CARMA(3,0)**

In [176]:
ar5 = np.array([3, 2.8, 0.8])
ma5 = np.array([1])

ar6 = np.array([3, 3.2, 1.2])
ma6 = np.array([1])

In [181]:
print('Real CARMA(3,0)\n')
print(f'Roots: {np.roots(np.append([1], ar5))}')
print(f'Hard Code ACF: {acf30(ar5, ma5)}')
print(f'Vector ACF: {acf(ar5, ma5)}')

# check terms given by CARMA solver
sigma, arpack, mapack = Kali2Pack(ar5, ma5)

print('\nTerms by CARMA Solver:')
carma_solver = CARMASolver(np.log(sigma), np.log(arpack), np.log(mapack))
carma_solver.get_celerite_coeffs()

Real CARMA(3,0)

Roots: [-1.4472136 -1.        -0.5527864]
Hard Code ACF: [ 0.17647188 -0.65789474  0.72813339]
Vector ACF: [ 0.17647188+0.j -0.65789474-0.j  0.72813339+0.j]

Terms by CARMA Solver:


(array([ 0.0288546 , -0.01168427,  0.00654108]),
 array([0.32296704, 2.47703296, 3.        ]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64))

In [185]:
print('Complex CARMA(3,0)\n')
print(f'Roots: {np.roots(np.append([1], ar6))}')
print(f'Hard Code ACF: {acf30(ar6, ma6)}')
print(f'Vector ACF: {acf(ar6, ma6)}')

# check terms given by CARMA solver
sigma, arpack, mapack = Kali2Pack(ar6, ma6)

print('\nTerms by CARMA Solver:')
carma_solver = CARMASolver(np.log(sigma), np.log(arpack), np.log(mapack))
carma_solver.get_celerite_coeffs()

Complex CARMA(3,0)

Roots: [-1.+0.4472136j -1.-0.4472136j -1.+0.j       ]
Hard Code ACF: [-0.22321429-1.66374105e-01j -0.22321429+1.66374105e-01j
  0.5952381 -1.96680666e-17j]
Vector ACF: [-0.22321429-0.16637411j -0.22321429+0.16637411j  0.5952381 +0.j        ]

Terms by CARMA Solver:


(array([ 0.01752493, -0.01796334,  0.01402918]),
 array([0.43380962, 2.76619038, 3.        ]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64))

<span style='color:red'>
The CARMA(3,0) test exposess more possible issues in the CARMA solver.
</span>

#### 2.4 Hard Code Celerite CARMAsolver in Python and compare with built-in

In [170]:
from scipy.special import logsumexp

# define CARMA parameters following carma_pack notation
sigma = 1
arpar = np.array([0.8, 2]) # low to high order
mapar = np.array([0.5])

arroots = np.roots(np.append([1], arpar[::-1]))
r1 = np.complex(arroots[0])
r2 = np.complex(arroots[1])

# let's only looking at the 1st Celerite term or j=1 case
term1 = np.log(sigma)
term2 = np.log(sigma)

term1 = logsumexp([term1, np.log(mapack[0])+np.log(r1)])
term2 = logsumexp([term2, np.log(mapack[0])+np.log(-r1)])

full_term = 2*np.log(sigma) + term1 + term2 - np.log(np.complex(-r1.real))
full_term -= (np.log(r2-r1) + np.log(np.conj(r2) + r1))
full_term = np.exp(full_term)
a_1 = 0.5*full_term.real
c_1 = -r1.real

# now the second Celerite term
term1 = np.log(sigma)
term2 = np.log(sigma)

term1 = logsumexp([term1, np.log(mapack[0])+np.log(r2)])
term2 = logsumexp([term2, np.log(mapack[0])+np.log(-r2)])

full_term = 2*np.log(sigma) + term1 + term2 - np.log(np.complex(-r2.real))
full_term -= (np.log(r1-r2) + np.log(np.conj(r1) + r2))
full_term = np.exp(full_term)
a_2 = 0.5*full_term.real
c_2 = -r2.real

print('Hand Code CARMA Solver for CARMA(2,1)')
print(f'a_1: {a_1}')
print(f'c_1: {c_1}')
print(f'a_2: {a_2}')
print(f'c_2: {c_2}')

print('\nSolution provided by Celerite CARMA Solver')
carma_solver = CARMASolver(np.log(sigma), np.log(arpar), np.log(mapar))
carma_solver.get_celerite_coeffs()

Hand Code CARMA Solver for CARMA(2,1)
a_1: -0.09200849718747373
c_1: 1.4472135954999579
a_2: 0.4670084971874737
c_2: 0.5527864045000421

Solution provided by Celerite CARMA Solver


(array([-0.11239837,  1.42489837]),
 array([0.5527864, 1.4472136]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64))

### 3. Find matches between CARMA ACF and Celerite terms
From the tests done above, we can see that vectorized ACF are quiet robust. We will start using that function only from now on.

#### 3.1 Test how np.roots order the roots
**3.1.1** 4 complex and 1 real roots

In [249]:
import numpy.polynomial.polynomial as poly

# 4 complex, 1 real
p1 = poly.Polynomial([1.2, 2, 1])
p2 = poly.Polynomial([1,1])
p3 = poly.Polynomial([1.1, 2, 1])
p4a = p1*p2*p3
print (f'Polynomical coeffs: {p4a.coef}') # this should from low to high
print (f'Roots from np.roots: \n{np.roots(p4a.coef[::-1])}')

Polynomical coeffs: [ 1.32  5.92 10.9  10.3   5.    1.  ]
Roots from np.roots: 
[-1.+0.4472136j  -1.-0.4472136j  -1.+0.31622777j -1.-0.31622777j
 -1.+0.j        ]


**3.1.2** 2 complex and 3 real roots

In [250]:
p1 = poly.Polynomial([0.8, 2, 1])
p2 = poly.Polynomial([1,1])
p3 = poly.Polynomial([1.1, 2, 1])
p4b = p1*p2*p3
print (f'Polynomical coeffs: {p4b.coef}') # this should from low to high
print (f'Roots from np.roots: \n{np.roots(p4b.coef[::-1])}')

Polynomical coeffs: [0.88 4.68 9.7  9.9  5.   1.  ]
Roots from np.roots: 
[-1.4472136+0.j         -1.       +0.31622777j -1.       -0.31622777j
 -1.       +0.j         -0.5527864+0.j        ]


#### 3.2 Function to return real/complex Celerite coeffs given ACF and roots

In [240]:
def get_real_coeffs(acf, roots):
    
    ar = []
    cr = []
    
    mask = np.iscomplex(roots)
    acf_real = acf[~mask]
    roots_real = roots[~mask]
    
    for i in range(len(acf_real)):
        ar.append(acf_real[i].real)
        cr.append(-roots_real[i].real)
    return ar, cr

def get_complex_coeffs(acf, roots):
    
    ac = []
    bc = []
    cc = []
    dc = []

    mask = np.iscomplex(roots)
    acf_complex = acf[mask]
    roots_complex = roots[mask]
    
    for i in range(len(acf_complex)):
        
        # only take every other root/acf
        if (i%2 == 0):            
            ac.append(2*acf_complex[i].real)
            bc.append(2*acf_complex[i].imag)
            cc.append(-roots_complex[i].real)
            dc.append(-roots_complex[i].imag)            
    
    return ac, bc, cc, dc


In [251]:
print('Real Coeffs for 1st example:')
print(get_real_coeffs(acf(p4a.coef[::-1][1:], [1]), np.roots(p4a.coef[::-1])))

print('\nComplex Coeffs for 1st example:')
print(get_complex_coeffs(acf(p4a.coef[::-1][1:], [1]), np.roots(p4a.coef[::-1])))

Real Coeffs for 1st example:
([1.4518002322881882], [1.0000000000001117])

Complex Coeffs for 1st example:
([0.6223972478715787, -1.9842440782801842], [1.1386820906581192, -1.8947972890619964], [1.0000000000000089, 0.9999999999999333], [-0.4472135955000447, -0.31622776601671737])


In [252]:
print('Real Coeffs for 2nd example:')
print(get_real_coeffs(acf(p4b.coef[::-1][1:], [1]), np.roots(p4b.coef[::-1])))

print('\nComplex Coeffs for 2nd example:')
print(get_complex_coeffs(acf(p4b.coef[::-1][1:], [1]), np.roots(p4b.coef[::-1])))

Real Coeffs for 2nd example:
([0.09660923971588471, -1.6046213093707336, 0.9665354598354214], [1.4472135954998717, 1.0000000000002485, 0.5527864045000291])

Complex Coeffs for 2nd example:
([0.7045361493508302], [0.7146219196211127], [0.999999999999925], [-0.31622776601678976])
